In [1]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV, LassoCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
RANDOM_STATE = 42

In [5]:
dataset = load_boston()
X = pd.DataFrame(dataset.data)
X.columns = dataset.feature_names
y = dataset.target

In [6]:
X.head(3)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03


In [7]:
y[:3]

array([24. , 21.6, 34.7])

1. Разделите выборку на обучающую и тестовую в отношении 80%/20%

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((404, 13), (102, 13), (404,), (102,))

2. Обучите стандартную регрессию, а также Ridge и  Lasso и параметрами по умолчанию и выведите их R2 на тестовой выборке

In [9]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f"R2: {r2:.3f}")

R2: 0.669


In [10]:
ridge = Ridge()
ridge.fit(X_train, y_train)

print (f'R2: {ridge.score(X_test, y_test):.3f}')

R2: 0.666


In [11]:
lasso = Lasso()
lasso.fit(X_train, y_train)
print (f'R2: {lasso.score(X_test, y_test):.3f}')

R2: 0.667


3. Для Ridge и Lasso подберите коэффициент регуляризации(используйте GridSearchCV, RidgeCV, LassoCV) в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по лучшим моделям и сравните с предыдущими результатами. Напишите как изменился результат

In [12]:
powers = []
for i in range(-5, 6):
    power = 10**(i)
    powers.append(power)

powers

[1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]

In [13]:
parameters = {'alpha': powers}

model = Ridge()
Ridge_reg= GridSearchCV(model, parameters, scoring='r2', cv=10)
Ridge_reg.fit(X_train, y_train)
print(Ridge_reg.best_estimator_)

best_model = Ridge_reg.best_estimator_
best_model.fit(X_train,y_train)

print (f'R2: {best_model.score(X_test, y_test):.3f}')

Ridge(alpha=1e-05)
R2: 0.669


In [14]:
ridgeCV = RidgeCV(alphas = powers)
ridgeCV.fit(X_train, y_train)
print (f'R2: {ridgeCV.score(X_test, y_test):.3f}')

R2: 0.669


In [ ]:
# результат через RidgeCV и GridsearchCV получается одинаковый
# результат немного лучше, чем для модели без регуляризации 

In [15]:
parameters = {'alpha': powers}

model = Lasso()
Lasso_reg= GridSearchCV(model, parameters, scoring='r2', cv=10)
Lasso_reg.fit(X_train, y_train)
print(Lasso_reg.best_estimator_)

best_model = Lasso_reg.best_estimator_
best_model.fit(X_train,y_train)

print (f'R2: {best_model.score(X_test, y_test):.3f}')

Lasso(alpha=1e-05)
R2: 0.669


In [16]:
model = LassoCV(alphas = powers)
model.fit(X_train, y_train)
print (f'R2: {model.score(X_test, y_test):.3f}')

R2: 0.669


In [ ]:
# результат через LassoCV и GridsearchCV получается одинаковый
# результат немного лучше, чем для модели без регуляризации 

4. Проведите масштабирование выборки(используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [17]:
standard_pipeline = Pipeline(steps=[
    ('scale', StandardScaler())
])

minmax_pipeline = Pipeline(steps=[
    ('scale', MinMaxScaler())
])

In [18]:
# Standard_Scaler

lr = LinearRegression()
lasso = Lasso()
ridge = Ridge()

lr_pipeline_s = Pipeline(steps=[
    ('preprocess', standard_pipeline),
    ('model', lr)
])

lasso_pipeline_s = Pipeline(steps=[
    ('preprocess', standard_pipeline),
    ('model', lasso)
])

ridge_pipeline_s = Pipeline(steps=[
    ('preprocess', standard_pipeline),
    ('model', ridge)
])

In [19]:
# MinMax_Scaler

lr_pipeline_m = Pipeline(steps=[
    ('preprocess', minmax_pipeline),
    ('model', lr)
])

lasso_pipeline_m = Pipeline(steps=[
    ('preprocess', minmax_pipeline),
    ('model', lasso)
])

ridge_pipeline_m = Pipeline(steps=[
    ('preprocess', minmax_pipeline),
    ('model', ridge)
])

In [20]:
lr_pipeline_s.fit(X_train, y_train)
lasso_pipeline_s.fit(X_train, y_train)
ridge_pipeline_s.fit(X_train, y_train)

y_pred_s = lr_pipeline_s.predict(X_test)
r2_s = r2_score(y_test, y_pred_s)

print('Standard Scaler')
print(f'Linear R2: {r2_s:.3f}')
print (f'Lasso R2: {lasso_pipeline_s.score(X_test, y_test):.3f}')
print (f'Ridge R2: {ridge_pipeline_s.score(X_test, y_test):.3f}')

Standard Scaler
Linear R2: 0.669
Lasso R2: 0.624
Ridge R2: 0.668


In [ ]:
# использование Standard_Scaler не повлияло на линейную модель, результаты Lasso и Ridge ухудшились

In [21]:
lr_pipeline_m.fit(X_train, y_train)
lasso_pipeline_m.fit(X_train, y_train)
ridge_pipeline_m.fit(X_train, y_train)

y_pred_m = lr_pipeline_m.predict(X_test)
r2_m = r2_score(y_test, y_pred_m)

print('Minmax Scaler')
print(f'Linear R2: {r2_m:.3f}')
print (f'Lasso R2: {lasso_pipeline_m.score(X_test, y_test):.3f}')
print (f'Ridge R2: {ridge_pipeline_m.score(X_test, y_test):.3f}')

Minmax Scaler
Linear R2: 0.669
Lasso R2: 0.257
Ridge R2: 0.676


In [ ]:
# использование MinMax_Scaler не повлияло на линейную модель, результаты Lasso и Ridge ухудшились (Lasso - значительно)

5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [22]:
parameters = {'alpha': powers}

pipe = Pipeline(steps=[
    ('scaler', 'passthrough'),
    ('model', 'passthrough')
])

parameters = [{
    'scaler': [StandardScaler(), MinMaxScaler()],
    'model': [Ridge()],
    'model__alpha': powers
}]

Ridge_reg= GridSearchCV(pipe, parameters, scoring='r2', cv=10, n_jobs = -1)
Ridge_reg.fit(X_train, y_train)

print('Ridge - best alpha:', Ridge_reg.best_params_['model__alpha'])

best_model = Ridge_reg.best_estimator_
best_model.fit(X_train,y_train)

print (f'R2: {best_model.score(X_test, y_test):.3f}')

Ridge - best alpha: 1
R2: 0.676


In [ ]:
# подбор параметров для Ridge позволил улучшить результат

In [23]:
parameters = {'alpha': powers}

pipe = Pipeline(steps=[
    ('scaler', 'passthrough'),
    ('model', 'passthrough')
])

parameters = [{
    'scaler': [StandardScaler(), MinMaxScaler()],
    'model': [Lasso()],
    'model__alpha': powers
}]

Lasso_reg= GridSearchCV(pipe, parameters, scoring='r2', cv=10, n_jobs = -1)
Lasso_reg.fit(X_train, y_train)

print('Lasso - best alpha:', Lasso_reg.best_params_['model__alpha'])

best_model = Lasso_reg.best_estimator_
best_model.fit(X_train,y_train)

print (f'R2: {best_model.score(X_test, y_test):.3f}')

Lasso - best alpha: 0.01
R2: 0.668


In [ ]:
# подбор параметров для Lasso дает результат, сопоставимый с линейной регрессией

6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [24]:
poly_pipeline_s = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('preprocessor', PolynomialFeatures(degree=2, include_bias=False)),
])

poly_pipeline_m = Pipeline(steps=[
    ('scaler', MinMaxScaler()),
    ('preprocessor', PolynomialFeatures(degree=2, include_bias=False)),
])

In [25]:
lr_pipeline_poly_s = Pipeline(steps=[
    ('preprocess', poly_pipeline_s),
    ('model',lr)
])

lasso_pipeline_poly_s = Pipeline(steps=[
    ('preprocess', poly_pipeline_s),
    ('model',lasso)
])

ridge_pipeline_poly_s = Pipeline(steps=[
    ('preprocess', poly_pipeline_s),
    ('model', ridge)
])

In [26]:
lr_pipeline_poly_m = Pipeline(steps=[
    ('preprocess', poly_pipeline_m),
    ('model',lr)
])

lasso_pipeline_poly_m = Pipeline(steps=[
    ('preprocess', poly_pipeline_m),
    ('model',lasso)
])

ridge_pipeline_poly_m = Pipeline(steps=[
    ('preprocess', poly_pipeline_m),
    ('model', ridge)
])

In [27]:
lr_pipeline_poly_s.fit(X_train, y_train)
lasso_pipeline_poly_s.fit(X_train, y_train)
ridge_pipeline_poly_s.fit(X_train, y_train)

y_pred_poly_s = lr_pipeline_poly_s.predict(X_test)
r2_poly_s = r2_score(y_test, y_pred_poly_s)

print('Polynomial features, Standard Scaler')
print(f'Linear R2: {r2_poly_s:.3f}')
print (f'Lasso R2: {lasso_pipeline_poly_s.score(X_test, y_test):.3f}')
print (f'Ridge R2: {ridge_pipeline_poly_s.score(X_test, y_test):.3f}')

Polynomial features, Standard Scaler
Linear R2: 0.805
Lasso R2: 0.732
Ridge R2: 0.816


In [ ]:
# использование полиномиальных фич и Standard_Scaler позволяет улучшить результат по всем видам моделей

In [28]:
lr_pipeline_poly_m.fit(X_train, y_train)
lasso_pipeline_poly_m.fit(X_train, y_train)
ridge_pipeline_poly_m.fit(X_train, y_train)

y_pred_poly_m = lr_pipeline_poly_m.predict(X_test)
r2_poly_m = r2_score(y_test, y_pred_poly_m)

print('Polynomial features, MinMax Scaler')
print(f'Linear R2: {r2_poly_m:.3f}')
print (f'Lasso R2: {lasso_pipeline_poly_m.score(X_test, y_test):.3f}')
print (f'Ridge R2: {ridge_pipeline_poly_m.score(X_test, y_test):.3f}')

Polynomial features, MinMax Scaler
Linear R2: 0.806
Lasso R2: 0.261
Ridge R2: 0.830


In [ ]:
# использование полиномиальных фич и MinMax_Scaler позволяет улучшить результат по линейной модели и Ridge
# результат для Lasso значительно ниже baseline

7. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2. Напишите как изменился R2 по сравнению с предыдущими экспериментами

In [29]:
pipe = Pipeline(steps=[
    ('scaler', 'passthrough'),
    ('preprocess', 'passthrough'),
    ('model', 'passthrough')
])

params = [{
    'scaler': [None, StandardScaler(), MinMaxScaler()],
    
    'preprocess': [None, 
             PolynomialFeatures(degree=2, include_bias=False),
             PolynomialFeatures(degree=3, include_bias=False),
             PolynomialFeatures(degree=4, include_bias=False)],
    
    'model': [Ridge(), Lasso()],
    'model__alpha': powers
}]

regression = GridSearchCV(pipe, params, scoring='r2', cv=10, n_jobs = -1)

regression.fit(X_train, y_train)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('scaler', 'passthrough'),
                                       ('preprocess', 'passthrough'),
                                       ('model', 'passthrough')]),
             n_jobs=-1,
             param_grid=[{'model': [Ridge(alpha=0.1), Lasso()],
                          'model__alpha': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1,
                                           10, 100, 1000, 10000, 100000],
                          'preprocess': [None,
                                         PolynomialFeatures(include_bias=False),
                                         PolynomialFeatures(degree=3,
                                                            include_bias=False),
                                         PolynomialFeatures(degree=4,
                                                            include_bias=False)],
                          'scaler': [None, StandardScaler(), MinMaxScaler()]}],
             scoring='r2'

In [30]:
print(regression.best_estimator_)

best_model = regression.best_estimator_

print (f'R2: {best_model.score(X_test, y_test):.3f}')

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('preprocess',
                 PolynomialFeatures(degree=4, include_bias=False)),
                ('model', Ridge(alpha=0.1))])
R2: 0.851


In [ ]:
# наилучшая модель при скоринге по R2 получается Ridge, c использованием полиномов и MinMax_Scaler
# получилось значительно улучшить результат по сравнению с моделями без дополнительной обработки признаков

http://archive.ics.uci.edu/ml/datasets/Adult

In [31]:
link = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/adult-all.csv'
data = pd.read_csv(link, header=None)

In [32]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import cross_val_score, KFold
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [33]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


8. Разделите выборку на признаки и целевую переменную(колонка со зачениями {<=50K,>50K}). Замените целевую переменную на числовые значения.

In [34]:
data[14].unique()

array(['<=50K', '>50K'], dtype=object)

In [35]:
di = {'<=50K': 0, '>50K': 1}
data[14] = data[14].map(di) 

In [36]:
X = data.iloc[:, 0:14]
y = data[14]

9. Выясните, присутствуют ли в данных пропуски. Заполните их самыми частыми значениями (испольуйте SimpleImputer)

In [37]:
X.isna().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
dtype: int64

In [ ]:
# пропусков нет

10. Выберите колонки с числовыми и категориальными переменными.

In [38]:
# числовые
X.select_dtypes(include= ['number']).columns.tolist()

[0, 2, 4, 10, 11, 12]

In [39]:
# категориальные
X.select_dtypes(include= ['object']).columns.tolist()

[1, 3, 5, 6, 7, 8, 9, 13]

In [40]:
numerical_ix = X.select_dtypes(include=['number']).columns
categorical_ix = X.select_dtypes(include=['object']).columns

11. Создайте пайплайн по обработке колонок(используйте OneHotEncoder,MinMaxScaler).

In [48]:
t = [('cat', OneHotEncoder(handle_unknown="ignore"), categorical_ix), 
     ('num', MinMaxScaler(), numerical_ix)]

col_transform = ColumnTransformer(transformers=t)

In [49]:
pipeline = Pipeline(steps=[('prep', col_transform), ('m', model)])

12. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

In [43]:
y.value_counts()

# самый частый класс - 0 (<=50K)

0    37155
1    11687
Name: 14, dtype: int64

In [44]:
preds = np.zeros(len(y))

In [45]:
print ('Accuracy: ', accuracy_score(y, preds))
print ('F1: ', f1_score(y, preds))

Accuracy:  0.7607182343065395
F1:  0.0


13. Посчитайте cross_val_score по алгоритмам LogisticRegression, SVC, LinearSVC по метрикам accuracy и f1_score.
Напишите удалось ли превзойти предыдущий результат.

In [52]:
model = LogisticRegression()

cv = KFold(n_splits=10, shuffle=True)
scores1 = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
scores2 = cross_val_score(pipeline, X, y, scoring='f1', cv=cv, n_jobs=-1)

print('Logistic Regression')
print ('Accuracy: ', scores1.mean())
print ('F1: ', scores2.mean())

Logistic Regression
Accuracy:  0.8516235077545211
F1:  0.656652400058673


In [ ]:
# использование логистической регрессии позволяет получить результат лучше, чем предсказание самого частого класса
# LogReg, SVC и LinearSVC дают сопоставимые результаты. По качеству LogReg находится на 2м месте.

In [53]:
model = svm.SVC()

cv = KFold(n_splits=10, shuffle=True)
scores1 = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
scores2 = cross_val_score(pipeline, X, y, scoring='f1', cv=cv, n_jobs=-1)

print('SVC')
print ('Accuracy: ', scores1.mean())
print ('F1: ', scores2.mean())

SVC
Accuracy:  0.8518486659172432
F1:  0.6568048951654273


In [ ]:
# использование SVC позволяет получить результат лучше, чем предсказание самого частого класса
# LogReg, SVC и LinearSVC дают сопоставимые результаты. По качеству SVC находится на 1м месте.

In [54]:
model = svm.LinearSVC()

cv = KFold(n_splits=10, shuffle=True)
scores1 = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
scores2 = cross_val_score(pipeline, X, y, scoring='f1', cv=cv, n_jobs=-1)

print('LinearSVC')
print ('Accuracy: ', scores1.mean())
print ('F1: ', scores2.mean())

LinearSVC
Accuracy:  0.8513368826163095
F1:  0.6572969769586605


In [ ]:
# использование LinearSVC позволяет получить результат лучше, чем предсказание самого частого класса
# LogReg, SVC и LinearSVC дают сопоставимые результаты. По качеству LinearSVC находится на 3м месте.

14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями (испольуйте SimpleImputer)

In [55]:
imputer = SimpleImputer(strategy='most_frequent', missing_values='?')
imputer = imputer.fit(X)
X.iloc[:,:] = imputer.transform(X)

15. Посчитайте cross_val_score на новых данных. Напишите удалось ли улучшить результат.

In [56]:
model = LogisticRegression()

cv = KFold(n_splits=10, shuffle=True)
scores1 = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
scores2 = cross_val_score(pipeline, X, y, scoring='f1', cv=cv, n_jobs=-1)

print('Logistic Regression')
print ('Accuracy: ', scores1.mean())
print ('F1: ', scores2.mean())

# результат немного ухудшился

Logistic Regression
Accuracy:  0.8512962259738103
F1:  0.6549104667898208


In [57]:
model = svm.SVC()

cv = KFold(n_splits=10, shuffle=True)
scores1 = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
scores2 = cross_val_score(pipeline, X, y, scoring='f1', cv=cv, n_jobs=-1)

print('SVC')
print ('Accuracy: ', scores1.mean())
print ('F1: ', scores2.mean())

# результат немного ухудшился

SVC
Accuracy:  0.8504360823091632
F1:  0.6544201832303146


In [58]:
model = svm.LinearSVC()

cv = KFold(n_splits=10, shuffle=True)
scores1 = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
scores2 = cross_val_score(pipeline, X, y, scoring='f1', cv=cv, n_jobs=-1)

print('LinearSVC')
print ('Accuracy: ', scores1.mean())
print ('F1: ', scores2.mean())

# результат немного ухудшился

LinearSVC
Accuracy:  0.850681769980644
F1:  0.6548727121282654


16. Посчитайте cross_val_score, если просто удалить значения '?'. Напишите как изменился результат

In [59]:
data_new = data[~(data == '?').any(axis=1)]

In [60]:
X = data_new.iloc[:, 0:14]
y = data_new[14]

In [61]:
model = LogisticRegression()

cv = KFold(n_splits=10, shuffle=True)
scores1 = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
scores2 = cross_val_score(pipeline, X, y, scoring='f1', cv=cv, n_jobs=-1)

print('Logistic Regression')
print ('Accuracy: ', scores1.mean())
print ('F1: ', scores2.mean())

# результат ухудшился по accuracy и улучшился по F1
# теперь LogReg на 3м месте по Accuracy и на 2м по F1

Logistic Regression
Accuracy:  0.8475079653328221
F1:  0.6607945464652485


In [62]:
model = svm.SVC()

cv = KFold(n_splits=10, shuffle=True)
scores1 = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
scores2 = cross_val_score(pipeline, X, y, scoring='f1', cv=cv, n_jobs=-1)

print('SVC')
print ('Accuracy: ', scores1.mean())
print ('F1: ', scores2.mean())

# результат ухудшился по accuracy и улучшился по F1
# теперь SVC на 2м месте по Accuracy и на 3м по F1

SVC
Accuracy:  0.8477508636138864
F1:  0.6604485259315263


In [63]:
model = svm.LinearSVC()

cv = KFold(n_splits=10, shuffle=True)
scores1 = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
scores2 = cross_val_score(pipeline, X, y, scoring='f1', cv=cv, n_jobs=-1)

print('LinearSVC')
print ('Accuracy: ', scores1.mean())
print ('F1: ', scores2.mean())

# результат ухудшился по accuracy и улучшился по F1
# теперь LinearSVC на 1м месте по обеим метрикам качества

LinearSVC
Accuracy:  0.8478395987367333
F1:  0.6617526155405502


 17. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier. Напишите как изменился результат и какой вывод можно из этого сделать.

In [64]:
X = data.iloc[:, 0:14]
y = data[14]

In [65]:
model = RandomForestClassifier()

cv = KFold(n_splits=10, shuffle=True)
scores1 = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
scores2 = cross_val_score(pipeline, X, y, scoring='f1', cv=cv, n_jobs=-1)

print('RandomForest')
print ('Accuracy: ', scores1.mean())
print ('F1: ', scores2.mean())

# результаты в целом сопоставимы с предыдущими алгоритмами, но RandomForest позволяет немного улучшить качество
# метрики RandomForest лучше, чем у GradientBoosting

RandomForest
Accuracy:  0.8512141498528397
F1:  0.6569180114541932


In [66]:
model = GradientBoostingClassifier()

cv = KFold(n_splits=10, shuffle=True)
scores1 = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
scores2 = cross_val_score(pipeline, X, y, scoring='f1', cv=cv, n_jobs=-1)

print('GradientBoosting')
print ('Accuracy: ', scores1.mean())
print ('F1: ', scores2.mean())

# результаты сопоставимы с предыдущими алгоритмами, но GradientBoosting позволяет немного улучшить качество

GradientBoosting
Accuracy:  0.8512140618333044
F1:  0.6565322436257277


18. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.

In [69]:
%%time
X = data.iloc[:, 0:14]
y = data[14]

Wall time: 16 ms


In [70]:
%%time
# оставляем пропуски как есть

X = data.iloc[:, 0:14]
y = data[14]

t1 = [('cat', OneHotEncoder(handle_unknown="ignore"), categorical_ix), 
     ('num', MinMaxScaler(), numerical_ix)]

t2 = [('cat', OneHotEncoder(handle_unknown="ignore"), categorical_ix), 
     ('num', StandardScaler(), numerical_ix)]

col_transform1 = ColumnTransformer(transformers=t1)
col_transform2 = ColumnTransformer(transformers=t2)

pipe = Pipeline(steps=[
    ('prep', 'passthrough'),
    ('model', 'passthrough')
])

params = [{
    'prep': [col_transform1, col_transform2],
    
    'model': [LogisticRegression(),
              svm.SVC(),
              svm.LinearSVC(),
              RandomForestClassifier(),
              GradientBoostingClassifier()            
            ]
}]

regression = GridSearchCV(pipe, params, scoring=['accuracy', 'f1'], refit = 'f1', cv=10, n_jobs = -1)

regression.fit(X, y)

Wall time: 23min 40s


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('prep', 'passthrough'),
                                       ('model', 'passthrough')]),
             n_jobs=-1,
             param_grid=[{'model': [LogisticRegression(), SVC(), LinearSVC(),
                                    RandomForestClassifier(),
                                    GradientBoostingClassifier()],
                          'prep': [ColumnTransformer(transformers=[('cat',
                                                                    OneHotEncoder(handle_unknown='ignore'),
                                                                    Int64Index([1, 3, 5, 6, 7, 8, 9, 13], dtype='int64')),
                                                                   ('num',
                                                                    MinMaxScaler(),
                                                                    Int64Index([0, 2, 4, 10, 11, 12], dtype='int64'))]),
                                

In [71]:
print(regression.best_estimator_)
best_model = regression.best_estimator_

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Int64Index([1, 3, 5, 6, 7, 8, 9, 13], dtype='int64')),
                                                 ('num', MinMaxScaler(),
                                                  Int64Index([0, 2, 4, 10, 11, 12], dtype='int64'))])),
                ('model', GradientBoostingClassifier())])


In [83]:
df_cv_scores=pd.DataFrame(regression.cv_results_).sort_values(by='rank_test_f1').reset_index()

df_cv_scores[['params','mean_test_f1', 'mean_test_accuracy', 'rank_test_f1']].head()

,params,mean_test_f1,mean_test_accuracy,rank_test_f1
0,"{'model': GradientBoostingClassifier(), 'prep'...",0.687155,0.867901,1
1,"{'model': GradientBoostingClassifier(), 'prep'...",0.687122,0.867880,2
2,"{'model': RandomForestClassifier(), 'prep': Co...",0.669056,0.853180,3
3,"{'model': RandomForestClassifier(), 'prep': Co...",0.668436,0.853057,4
4,"{'model': SVC(), 'prep': ColumnTransformer(tra...",0.663146,0.857193,5


In [73]:
print ('Accuracy: ', df_cv_scores.loc[0,'mean_test_accuracy'])
print ('F1: ', df_cv_scores.loc[0,'mean_test_f1'])

Accuracy:  0.867900654446202
F1:  0.6871548063849853


In [ ]:
# науилучший результат получается при использовании GradientBoosting, не используя замены/удаления пропусков (см.ниже)
# для категориальных признаков используем OneHotEncoder, для числовых - MinMax_Scaler

In [74]:
# заменям пропуски на most_frequent

X = data.iloc[:, 0:14]
y = data[14]

imputer = SimpleImputer(strategy='most_frequent', missing_values='?')
imputer = imputer.fit(X)
X.iloc[:,:] = imputer.transform(X)


t1 = [('cat', OneHotEncoder(handle_unknown="ignore"), categorical_ix), 
     ('num', MinMaxScaler(), numerical_ix)]

t2 = [('cat', OneHotEncoder(handle_unknown="ignore"), categorical_ix), 
     ('num', StandardScaler(), numerical_ix)]

col_transform1 = ColumnTransformer(transformers=t1)
col_transform2 = ColumnTransformer(transformers=t2)

pipe = Pipeline(steps=[
    ('prep', 'passthrough'),
    ('model', 'passthrough')
])

params = [{
    'prep': [col_transform1, col_transform2],
    
    'model': [LogisticRegression(),
              svm.SVC(),
              svm.LinearSVC(),
              RandomForestClassifier(),
              GradientBoostingClassifier()            
            ]
}]

regression2 = GridSearchCV(pipe, params, scoring=['accuracy', 'f1'], refit = 'f1', cv=10, n_jobs = -1)

regression2.fit(X, y)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('prep', 'passthrough'),
                                       ('model', 'passthrough')]),
             n_jobs=-1,
             param_grid=[{'model': [LogisticRegression(), SVC(), LinearSVC(),
                                    RandomForestClassifier(),
                                    GradientBoostingClassifier()],
                          'prep': [ColumnTransformer(transformers=[('cat',
                                                                    OneHotEncoder(handle_unknown='ignore'),
                                                                    Int64Index([1, 3, 5, 6, 7, 8, 9, 13], dtype='int64')),
                                                                   ('num',
                                                                    MinMaxScaler(),
                                                                    Int64Index([0, 2, 4, 10, 11, 12], dtype='int64'))]),
                                

In [82]:
df_cv_scores2=pd.DataFrame(regression2.cv_results_).sort_values(by='rank_test_f1')

df_cv_scores2[['params','mean_test_f1', 'mean_test_accuracy', 'rank_test_f1']].head()

,params,mean_test_f1,mean_test_accuracy,rank_test_f1
8,"{'model': GradientBoostingClassifier(), 'prep'...",0.681732,0.866242,1
9,"{'model': GradientBoostingClassifier(), 'prep'...",0.681698,0.866222,2
6,"{'model': RandomForestClassifier(), 'prep': Co...",0.667265,0.852299,3
7,"{'model': RandomForestClassifier(), 'prep': Co...",0.666443,0.852115,4
3,"{'model': SVC(), 'prep': ColumnTransformer(tra...",0.660625,0.856517,5


In [84]:
print(regression2.best_estimator_)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Int64Index([1, 3, 5, 6, 7, 8, 9, 13], dtype='int64')),
                                                 ('num', MinMaxScaler(),
                                                  Int64Index([0, 2, 4, 10, 11, 12], dtype='int64'))])),
                ('model', GradientBoostingClassifier())])


In [ ]:
# науилучший результат получается при использовании GradientBoosting
# замена пропусков на most_frequent немного снижает качество
# для категориальных признаков используем OneHotEncoder, для числовых - MinMax_Scaler

In [78]:
# удаляем пропуски

X_new = data_new.iloc[:, 0:14]
y_new = data_new[14]

t1 = [('cat', OneHotEncoder(handle_unknown="ignore"), categorical_ix), 
     ('num', MinMaxScaler(), numerical_ix)]

t2 = [('cat', OneHotEncoder(handle_unknown="ignore"), categorical_ix), 
     ('num', StandardScaler(), numerical_ix)]

col_transform1 = ColumnTransformer(transformers=t1)
col_transform2 = ColumnTransformer(transformers=t2)

pipe = Pipeline(steps=[
    ('prep', 'passthrough'),
    ('model', 'passthrough')
])

params = [{
    'prep': [col_transform1, col_transform2],
    
    'model': [LogisticRegression(),
              svm.SVC(),
              svm.LinearSVC(),
              RandomForestClassifier(),
              GradientBoostingClassifier()            
            ]
}]

regression3 = GridSearchCV(pipe, params, scoring=['accuracy', 'f1'], refit = 'f1', cv=10, n_jobs = -1)

regression3.fit(X, y)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('prep', 'passthrough'),
                                       ('model', 'passthrough')]),
             n_jobs=-1,
             param_grid=[{'model': [LogisticRegression(), SVC(), LinearSVC(),
                                    RandomForestClassifier(),
                                    GradientBoostingClassifier()],
                          'prep': [ColumnTransformer(transformers=[('cat',
                                                                    OneHotEncoder(handle_unknown='ignore'),
                                                                    Int64Index([1, 3, 5, 6, 7, 8, 9, 13], dtype='int64')),
                                                                   ('num',
                                                                    MinMaxScaler(),
                                                                    Int64Index([0, 2, 4, 10, 11, 12], dtype='int64'))]),
                                

In [81]:
df_cv_scores3=pd.DataFrame(regression3.cv_results_).sort_values(by='rank_test_f1')

df_cv_scores3[['params','mean_test_f1', 'mean_test_accuracy', 'rank_test_f1']].head()

,params,mean_test_f1,mean_test_accuracy,rank_test_f1
8,"{'model': GradientBoostingClassifier(), 'prep'...",0.681763,0.866242,1
9,"{'model': GradientBoostingClassifier(), 'prep'...",0.681668,0.866222,2
7,"{'model': RandomForestClassifier(), 'prep': Co...",0.671646,0.854551,3
6,"{'model': RandomForestClassifier(), 'prep': Co...",0.668899,0.853139,4
3,"{'model': SVC(), 'prep': ColumnTransformer(tra...",0.660625,0.856517,5


In [85]:
print(regression3.best_estimator_)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Int64Index([1, 3, 5, 6, 7, 8, 9, 13], dtype='int64')),
                                                 ('num', MinMaxScaler(),
                                                  Int64Index([0, 2, 4, 10, 11, 12], dtype='int64'))])),
                ('model', GradientBoostingClassifier())])


In [ ]:
# науилучший результат получается при использовании GradientBoosting
# удаление пропусков немного снижает качество
# для категориальных признаков используем OneHotEncoder, для числовых - MinMax_Scaler